<a href="https://colab.research.google.com/github/GrigoryBartosh/hse08_ip/blob/master/hw4_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd 'drive/My Drive/ip_hw_4'

/content/drive/My Drive/ip_hw_4


In [0]:
import os
import copy
import zipfile
import time

import numpy as np
from skimage.segmentation import slic, mark_boundaries
from sklearn.cluster import MeanShift
import cv2

from PIL import Image

from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt
import matplotlib.cm as cm

PATH_DATASET = 'leaves.zip'
PATH_RESULTS = 'results.zip'

In [6]:
images, image_names = [], []
with zipfile.ZipFile(PATH_DATASET, 'r') as zip_file:
    file_paths = zip_file.namelist()
    file_paths = list(filter(lambda f: f.endswith('jpg'), file_paths))

    for file_path in tqdm(file_paths):
        with zip_file.open(file_path) as img_file:
            image = Image.open(img_file).convert('RGB')
            image = np.array(image, dtype=np.float32)

            images += [image]
            image_names += [file_path.split('/')[1]]

In [0]:
foo_cvt = lambda img: cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2LAB)
lab_images = [foo_cvt(img) for img in images]
lab_images = [img.astype(np.float32) for img in lab_images]

centers = [img[img.shape[0] // 2, img.shape[1] // 2] for img in lab_images]
centers = np.stack(centers, axis=0)
center_color = centers.mean(axis=0)

dist_images = [abs(img - center_color) for img, c in zip(lab_images, centers)]

In [0]:
def do_segmentation(image, dist_image):
    image = cv2.bilateralFilter(image, 15, 80, 80)

    segments = slic(image, n_segments=8, sigma=9)

    best_i = -1
    best_mean = 1000
    for i in np.unique(segments):
        mask = (segments == i)
        l_mean = dist_image[mask].mean()
        if best_mean > l_mean:
            best_mean = l_mean
            best_i = i

    res = np.zeros_like(image, dtype=np.uint8)
    mask = (segments == best_i)
    res[mask] = 255

    return res

In [288]:
segmented_images = [do_segmentation(img, dimg) 
                    for img, dimg in tqdm(zip(images, dist_images))]

In [289]:
if os.path.exists(PATH_RESULTS):
    os.remove(PATH_RESULTS)

with zipfile.ZipFile(PATH_RESULTS, 'w') as zip_file:
    for image_name, image in tqdm(zip(image_names, segmented_images)):
        with zip_file.open(image_name, 'w') as img_file:
            image = Image.fromarray(image.astype(np.uint8))
            image.save(img_file, 'JPEG', quality=100)